In [20]:
import os
import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout
# from PIL import ImageFile
from keras.applications.resnet50 import ResNet50
import shutil
from keras.models import model_from_json

In [21]:
DataPaths=pd.read_csv('AllData2.csv')

In [22]:
Classes=pd.read_csv('DataStats2.csv')

In [24]:
def build_ResNet50(input_tensor_shape,output_shape):
    '''
    # reference 
        https://keras.io/applications/#vgg16
        https://www.tensorflow.org/api_docs/python/tf/contrib/keras/applications/ResNet50
    # model defination
        https://github.com/tensorflow/tensorflow/blob/r1.2/tensorflow/contrib/keras/python/keras/applications/resnet50.py
        
    # Arguments
        include_top: whether to include the fully-connected layer at the top of the network.
     
    '''
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape= input_tensor_shape)
    
    x_model = base_model.output
    
    x_model = GlobalAveragePooling2D(name='globalaveragepooling2d')(x_model)
    
    #x_model = Dense(1024, activation='relu',name='fc1_Dense')(x_model)
    #x_model = Dropout(0.5, name='dropout_1')(x_model)
    
    x_model = Dense(256, activation='relu',name='fc2_Dense')(x_model)
    x_model = Dropout(0.5, name='dropout_2')(x_model)
    predictions = Dense(output_shape, activation='sigmoid',name='output_layer')(x_model)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model


In [25]:
def SaveModel(model,Path):
    # serialize model to JSON
    model_json = model.to_json()
    if os.path.isdir(Path):
        shutil.rmtree(Path)
    os.mkdir(Path)
    with open(Path+"/model_architecture.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(Path+"/model_weights.h5")
    print("Saved model to ",Path)

def loadModel(Path):
    # load json and create model
    json_file = open(Path+'/model_architecture.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(Path+"/model_weights.h5")
    print("Loaded model from disk",Path)
    return loaded_model

In [26]:
# get model
img_rows, img_cols, img_channel = 224, 224, 3
input_tensor_shape=(img_rows, img_cols, img_channel)
output_shape=Classes.shape[0]
model = build_ResNet50(input_tensor_shape,output_shape)

In [27]:
SaveModel(model,'Resnet50/pretrained3')

('Saved model to ', 'Resnet50/pretrained3')


In [28]:
pwd

u'/Users/kumarshubham/Desktop/COP_Project'

In [29]:
print len(model.layers)
model.summary()

179
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activa

In [44]:
model=loadModel('Resnet_50/Pretrained')

('Loaded model from disk', 'Resnet50/Pretrained')


In [36]:
#FreezeLayers
for layer in model.layers[:175]:
        layer.trainable = False
for layer in model.layers[175:]:
    layer.trainable = True

In [19]:
from direct.showbase.ShowBase import ShowBase
from direct.task import Task
from math import pi, sin, cos
from direct.actor.Actor import Actor
from direct.interval.IntervalGlobal import Sequence
from panda3d.core import Point3